<a href="https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/WOS_SCI_SCP_PTJ_CTR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WOS+SCI+SCP+PTJ+CTR

Merge the bibliographic datasets for 
* Web of Science (WOS), 
* Scielo (SCI)
* Scopus  (SCP)
* Puntaje (UDEA)
* Center (CTR)
of the scientific articles of Universidad de Antioquia

For details see [merge.ipynb in Colaboratory](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/merge.ipynb)

Implementation:
The input pure o partially processed database with WOS-SCI-SCP and may be some UDEA entries from PTJ and Center information with additional data about the Full Name UDEA authors.

Addtionaly UDEA entries can be captured from:
1. A previous WOS-SCI-SCP-UDEA
2. A Data Base with a column with full names (FULL LAST NAMES NAMES, e.g VALDEZ GÚZMAN JUAN ALBERTO) and a list of author Aliases in WOS format (Lastname, Name, e.g Valdez-Gúzman, J.A.) with a list of registered affiliations. TODO: Test
3. The database from Puntaje (UDEA). 

In [1]:
import os
VERSION='NEW'
if os.getcwd()=='/content':
    !pip install openpyxl xlrd wosplus fuzzywuzzy[speedup] > /dev/null

In [2]:
# Delete UDEA_columns and start from schratch
REBUILD=False
MERGE_WITH_TRAINED=True

## functions

In [3]:
import pandas as pd
import wosplus as wp
pd.set_option('display.max_colwidth',200)

In [4]:
# %load wos_sci_scp_ptj_ctr.py

In [5]:
from wos_sci_scp_ptj_ctr import *

##  Configure public links of  files in Google Drive
* If it is a Google Spreadsheet the corresponding file is downloaded as CSV
* If it is in excel or text file the file is downloaded  directly

To define your  own labeled IDs for public google drive files edit the next cell:

In [6]:
%%writefile drive.cfg
[FILES]
WOS_SCI_SCP_PTJ_CTR.json.gz=19E1C1kRk4I0V3uXojqko8-NEicWaPp1j
WOS_SCP_UDEA_SJR_SIU.xlsx=0BxoOXsn2EUNIQ3R4WDhvSzVLQ2s
Base_de_datos_investigadores_Definitiva.csv=12oalgUeKhpvzkTPBP8pXCeHTrF-KO223dy9ov9w9QKs
UDEA_authors_with_WOS_info.json=1o1eVT4JD0FMMICq_oxrTJOzWh47veBMw
produccion_fecha_vig_2003_2018.xlsx=1WbtX4K__TTLxXRjuLvqUYz9tuHCIlS5v
UDEA_WOS_SCI_SCP_PTJ.json=1OkVytKbxJwGvXZDkynkSoUDtkUOTaT4A

Overwriting drive.cfg


##  Load data bases

In [7]:
affil='Univ Antioquia'
drive_files=wp.wosplus('drive.cfg')

#### DEBUG: if False stop in UDEA_PTJ!!!!

if os.path.exists(UDEAjsonfile):
    UDEA=               pd.read_json(UDEAjsonfile,compression='gzip').reset_index(drop=True)
else:    
    UDEA=drive_files.read_drive_json(UDEAjsonfile,compression='gzip').reset_index(drop=True)

In [8]:
if REBUILD:
    !rm WOS_SCI_SCP_PTJ_CTR.json.gz

In [9]:
RECOVER=True #False for test purposes
UDEAjsonfile='WOS_SCI_SCP_PTJ_CTR.json.gz'
#Test purposes
#UDEAjsonfile='UDEA_WOS_SCI_SCP_PTJ.json'
if RECOVER:
    #Requieres latest wosplus!
    tmp=drive_files.load_biblio(UDEAjsonfile,compression='gzip')# TODO CHANGE FOR LAST VERSION IN GOOGLE DRIVE
else:
    tmp=drive_files.load_biblio('UDEAtmp.json')
    #drive_files.load_biblio(
    #  'https://raw.githubusercontent.com/restrepo/medicion/master/cienciometria/data/UDEAtmp300.json'
    #    )#Test: 199+1=200 found
    
UDEA=drive_files.biblio['WOS'].reset_index(drop=True)
#DEBUG
#UDEA=UDEA.sample(300,replace=True).reset_index(drop=True) #Test: 77 found
#tmp=drive_files.load_biblio('Sample_WOS.xlsx')

In [10]:
df=UDEA
UDEA_authors='UDEA_authors'
kk=df[UDEA_authors].apply(lambda l:
             l if type(l)==list
             and len(l)>0 else None
                ).dropna().reset_index(drop=True)

kk[kk.apply(
    lambda l: len([1 for d in l if 
               d.get('full_name')])>0
       )].shape

(11681,)

In [11]:
kk[kk.apply(
    lambda l: len([1 for d in l if 
               d.get('NOMBRE COMPLETO')])>0
       )].shape

(7717,)

In [12]:
df=UDEA
Tipo='Tipo'
x=df[df[Tipo].str.contains(
      'UDEA')].shape[0]
print(x)

10311


In [13]:
%%writefile check_quality.py
def check_quality(df,
     authors_WOS='authors_WOS',
     Tipo='Tipo',
     UDEA_authors='UDEA_authors'
    ):
    import pandas as pd
    if authors_WOS in df.columns:
        print(authors_WOS)
        x=df[authors_WOS].apply(lambda l:
                 l if type(l)==list
                 and len(l)>0 else None
                    ).dropna().shape[0]
        print(x)
        kk=df[df['TI']=='Leptonic charged Higgs decays in the Zee model'].reset_index(drop=True)
        print(kk.loc[0,'TI'],'; authors_WOS:',kk.loc[0,authors_WOS],'; AU:',kk.loc[0,'AU'])
    if Tipo in df.columns:        
        print('Tipo contains UDEA')
        x=df[df[Tipo].str.contains(
             'UDEA')].shape[0]
        print(x)
    if UDEA_authors in df.columns:
        print(UDEA_authors)
        kk=df[UDEA_authors].apply(lambda l:
             l if type(l)==list
             and len(l)>0 else None
                ).dropna().reset_index(drop=True)
        print(kk.shape[0])

        print('UDEA_authors → full_names (Extrapolado puntaje)')
        x=kk[kk.apply(lambda l: len([1 for d in l if 
                   d.get('full_name')])>0
               )].shape[0]
        print(x)
    
        print('UDEA_authors → "NOMBRE COMPLETO" (Extrapolado CENTRO)')
        x=kk[kk.apply(lambda l: len([1 for d in l if 
                   d.get('NOMBRE COMPLETO')])>0
               )].shape[0]
        print(x)

Overwriting check_quality.py


In [14]:
from check_quality import *
check_quality(UDEA)

authors_WOS
13645
Leptonic charged Higgs decays in the Zee model ; authors_WOS: [] ; AU: Sierra, DA
Restrepo, D

Tipo contains UDEA
10311
UDEA_authors
11681
UDEA_authors → full_names (Extrapolado puntaje)
11681
UDEA_authors → "NOMBRE COMPLETO" (Extrapolado CENTRO)
7717


In [15]:
if REBUILD:
    UDEA=clean_institutional_columns(UDEA,prefix='UDEA',Tipo='Tipo')
    UDEA['UDEA_authors']=None


In [16]:
for t in UDEA.Tipo.unique():
    print( '{}:{}'.format( t, UDEA[ UDEA.Tipo==t].shape[0] ) )

WOS_SCP_UDEA:4468
WOS_UDEA:716
WOS_SCI_SCP_UDEA:599
SCI_SCP_UDEA:1269
WOS_SCI_UDEA:93
SCI_UDEA:2082
SCP_UDEA:1084
WOS:1168
SCP:1489
WOS_SCP:1352
WOS_SCI_SCP:169
SCI_SCP:347
WOS_SCI:54
SCI:810


In [17]:
UDEA.shape

(15700, 181)

## Load trained old data 

### Merge WOS_SCP_SCI with trained data set PTJ_CTR

Merge requires split in DI and TI


15700 (15700, 152)
(7072, 169) (8628, 169)

In [18]:
SIU=drive_files.read_drive_excel('WOS_SCP_UDEA_SJR_SIU.xlsx')

In [19]:
if MERGE_WITH_TRAINED:
    if os.path.exists('WOS_SCP_UDEA_SJR_SIU.xlsx'):
        SIU=pd.read_excel('WOS_SCP_UDEA_SJR_SIU.xlsx')
    else:    
        SIU=drive_files.read_drive_excel('WOS_SCP_UDEA_SJR_SIU.xlsx')
        
    UDEA,SIU=fill_trained_data(UDEA,SIU)#TODO: Remnove SIU

/usr/local/lib/python3.5/dist-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


15700 (15700, 152)
(7072, 168) (8628, 168)


In [20]:
if MERGE_WITH_TRAINED:
    UDEA.to_json('UDEAtmp.json')
    RECOVER=False
    if RECOVER:
        UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

In [21]:
if 'UDEA_autores' in UDEA.columns and UDEA[UDEA['UDEA_autores']==''].shape[0]:
    UDEA['UDEA_autores']=UDEA['UDEA_autores'].apply(lambda s: pd.np.nan if type(s)==str and s=='' else s)

In [22]:
if 'UDEA_autores' in UDEA.columns:
    print(UDEA[UDEA['UDEA_autores']==''].shape[0],UDEA['UDEA_autores'].dropna().shape[0])

0 7072


In [23]:
check_quality(UDEA)

authors_WOS
13645
Leptonic charged Higgs decays in the Zee model ; authors_WOS: [] ; AU: Sierra, DA
Restrepo, D

Tipo contains UDEA
10311


# Puntaje

UDEA

In [24]:
qq=UDEA.copy()

In [25]:
drive_files.biblio['WOS']=qq
drive_files.biblio['WOS'].shape

(15700, 168)

In [26]:
tmp=drive_files.load_biblio('produccion_fecha_vig_2003_2018.xlsx',prefix='UDEA')

In [27]:
pp= drive_files.biblio['UDEA'].copy()

In [28]:
drive_files.biblio['UDEA']=pp

In [29]:
df=merge_puntaje(drive_files)

(32581, 24)
va1 0 0
........................................................................

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


7258 : 5806 + 2827 = 8633
va2 0 5806
.........................................................

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


7258 : 5453 + 353 = 5806
va3 0 5453
.......................................................

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


va4 0 5389
7258 : 5389 + 64 = 5453
(3239, 174) + (5389, 152) = 8628


In [30]:
#TODO: Check why not zero
if 'UDEA_autores' in df.columns:
    print(0,'=',df[df['UDEA_autores']==''].shape[0],'; found:',df['UDEA_autores'].dropna().shape[0])

0 = 0 ; found: 10311


In [31]:
#df['UDEA_autores'].apply(lambda s: pd.np.nan if type(s)==str and s=='' else s).dropna().shape

In [32]:
UDEA=df.copy()

In [33]:
UDEA.shape

(15700, 180)

In [34]:
check_quality(UDEA)

authors_WOS
13645
Leptonic charged Higgs decays in the Zee model ; authors_WOS: [] ; AU: Sierra, DA
Restrepo, D

Tipo contains UDEA
10311


## Fill C1 for not WOS entries in WOS format and extract  affiliation from C1

In [35]:
#Fill from SCI_C1
UDEA['C1']=SCI_C1_to_C1(UDEA)

In [36]:
#Fill from SCP_C1='SCP_Authors with affiliations
UDEA['C1']=SCP_Authors_with_affiliations_to_C1(UDEA)

In [37]:
UDEA[UDEA['C1'].isnull()].shape

(0, 180)

In [38]:
UDEA[UDEA.Tipo=='WOS'].reset_index(drop=True).C1.loc[0]

'[Lenis, Yasser Y.; Johnson, Gregory A.; Wang, Xiaoqiu; Tang, Wendy W.; Dunlap, Kathrin A.; Satterfield, M. Carey; Wu, Guoyao; Bazer, Fuller W.] Texas A&M Univ, Dept Anim Sci, College Stn, TX 77843 USA.\n[Lenis, Yasser Y.; Johnson, Gregory A.; Wang, Xiaoqiu; Tang, Wendy W.; Dunlap, Kathrin A.; Wu, Guoyao; Bazer, Fuller W.] Texas A&M Univ, Ctr Anim Biotechnol & Genom, College Stn, TX 77843 USA.\n[Lenis, Yasser Y.] Univ Antioquia, Sch Vet Med, Fac Agrarian Sci, Centauro Res Grp, Calle 70 52-21, Medellin, Colombia.\n[Lenis, Yasser Y.] UDCA, Fac Agr Sci, Calle 222 55-37, Bogota, Colombia.\n[Wang, Xiaoqiu] NIEHS, POB 12233, Res Triangle Pk, NC 27709 USA.\n[Hansen, Thomas R.] Colorado State Univ, Coll Vet Med & Biomed Sci, Anim Reprod & Biotechnol Lab Dept Biomed Sci, Ft Collins, CO 80523 USA.\n'

In [39]:
#WARNING: some C1 WOS entries are not normalized: Missing authors
UDEA['authors_WOS']=UDEA.C1.apply(lambda x: x.split('\n') if x else x).apply(
    lambda x:   [y.replace('[','').replace('] ','; ') for y in x if y.find(affil)>-1 ] if x else x ).apply(
     lambda x: get_author_info(x) if x else x)

# Improve normalization: remove C1s with only affiliation (from Scielo)
UDEA['authors_WOS']=UDEA['authors_WOS'].apply( 
    lambda x: [d for d in x if d.get('WOS_author').find(affil)==-1] if type(x)==list else x )

In [40]:
UDEA[UDEA.Tipo=='SCP'].reset_index(drop=True).loc[0].authors_WOS

[{'WOS_author': 'Cadavid, Y.',
  'affiliation': ['Grupo de Ciencia y Tecnología del Gas y Uso Racional de la Energía, Facultad de Ingeniería, Univ Antioquia, Calle 67 No 53-108, Bloque 19-000, Medellín, Colombia'],
  'i': 0},
 {'WOS_author': 'Amell, A.',
  'affiliation': ['Grupo de Ciencia y Tecnología del Gas y Uso Racional de la Energía, Facultad de Ingeniería, Univ Antioquia, Calle 67 No 53-108, Bloque 19-000, Medellín, Colombia'],
  'i': 1}]

## Prepare UDEA columns

In [41]:
#TODO: Remove from fill_trained_data(..)
if 'UDEA_autores' in UDEA.columns:
    UDEA['UDEA_autores']=UDEA['UDEA_autores'].apply(lambda s: re.sub('\s+',' ',s) if type(s)==str else s)
    UDEA['UDEA_authors']=UDEA['UDEA_autores'].apply(lambda s: s.split(';') if type(s)==str else s).apply(
                           lambda l: [{'full_name':y} for y in l ] if type(l)==list else l)

## Merge with official researcher list: PTJ

In [42]:
AU=drive_files.read_drive_excel('Base_de_datos_investigadores_Definitiva.csv')

In [43]:
UPDATE_UDEA_authors_with_AU=True
if (UDEA['UDEA_authors'].dropna().shape[0] and 
    UPDATE_UDEA_authors_with_AU):
    kkn=UDEA.copy()
    kkn=update_institutional_authors(kkn,AU)
    print(kkn.shape,UDEA.shape)
    UDEA=kkn.copy()

0
1
2
3
4
5
6
7
8
9
10
11
(15700, 181) (15700, 181)


Quality check

In [44]:
key_contains_in_list_of_dictionaries(UDEA,'Restrepo, D',column='authors_WOS',key='WOS_author').loc[1:2]

1    [{'WOS_author': 'Restrepo, Diego', 'i': 0, 'affiliation': ['Univ Antioquia, Inst Fis, Medellin 1226, Colombia.']}, {'WOS_author': 'Rivera, Andres', 'i': 1, 'affiliation': ['Univ Antioquia, Inst Fi...
2                                                                                                   [{'WOS_author': 'Restrepo, D.', 'i': 0, 'affiliation': ['Univ Antioquia, Inst Fis, Bogota, Colombia.']}]
Name: authors_WOS, dtype: object

In [45]:
if UPDATE_UDEA_authors_with_AU:
    UDEA.to_json('UDEAtmp.json')
    RECOVER=False
    if RECOVER:
        UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

In [46]:
check_quality(UDEA)

authors_WOS
13645
Leptonic charged Higgs decays in the Zee model ; authors_WOS: [] ; AU: Sierra, DA
Restrepo, D

Tipo contains UDEA
10311
UDEA_authors
10311
UDEA_authors → full_names (Extrapolado puntaje)
10311
UDEA_authors → "NOMBRE COMPLETO" (Extrapolado CENTRO)
6840


## Add `UDEA.authors_WOS` info* within `UDEA.UDEA_authors` data**
(\*) obtained from `UDEA.C1`

(\*\*) Obtained from [puntaje trained old UDEA data](./WOS_SCI_SCP_PTJ_GS_LNS.ipynb#Merge-with-trained-data-set) and the [official researcher list](./WOS_SCI_SCP_PTJ_GS_LNS.ipynb#Merge-with-official-researcher-list)

Obtain name parts and initials from full name in `UDEA_authors` dictionary and update `UDEA_authors` with them

In [47]:
import sys
if 'UDEA_authors' not in UDEA.columns and REBUILD==False:
    sys.exit('Make MERGE_WITH_TRAINED True and run again')

In [48]:
# Obtain spanish name parts from full name
dictupdatetmp=UDEA['UDEA_authors'].apply(lambda x: [y.update( 
                split_full_names(y,full_name='full_name')  ) if not pd.isnull(
                y.get('full_name')) else y for y in x] 
                                   if type(x)==list 
                                   else x)

In [49]:
kk=UDEA['authors_WOS'].combine( UDEA['UDEA_authors'], func=combinewos )

In [51]:
UDEA.to_json('UDEAtmp.json')

### Load output restuls of previous Cell runs

In [52]:
RECOVER=False
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

## Build a single profile for all

### Fill UDEA_authors with WOS_author info

Obtain UDEA_authors DataFrame: `aunly`

In [53]:
aunly=DataFrame_authors(UDEA)

DELGADO LASTRA JUAN DE DIOS


In [54]:
if not aunly.empty:
    aunly.to_json('UDEA_authors_with_WOS_info.json')

In [55]:
RECOVER=False
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

In [56]:
UDEA.shape

(15700, 181)

In [57]:
if RECOVER:
    if os.path.exists('UDEA_authors_with_WOS_info.json' ):
        aunly=pd.read_json('UDEA_authors_with_WOS_info.json')
    else:
        aunly=drive_files.read_drive_json('UDEA_authors_with_WOS_info.json')

In [58]:
aunly.shape

(1273, 2)

(1273, 2)

In [59]:
check_quality(UDEA)

authors_WOS
13645
Leptonic charged Higgs decays in the Zee model ; authors_WOS: [] ; AU: Sierra, DA
Restrepo, D

Tipo contains UDEA
10311
UDEA_authors
10311
UDEA_authors → full_names (Extrapolado puntaje)
10311
UDEA_authors → "NOMBRE COMPLETO" (Extrapolado CENTRO)
6840


## Merge UDEA with authors

With exact author matching and high `lv.ratio` for affiliation

In [60]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(lambda l:fill_full_wos_author_info(l,aunly) )

In [61]:
if UDEA['UDEA_authors'].dropna().shape[0]:
    UDEA.to_json('UDEAtmp.json')

In [62]:
RECOVER=False
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

In [63]:
UDEA.shape

(15700, 181)

In [64]:
kk=UDEA.authors_WOS.combine(UDEA.UDEA_authors,func=lambda x,y: get_UDEA_authors(x,y,aunly))

In [65]:
UDEA.UDEA_authors.dropna().shape

(10311,)

(7072,)

In [66]:
UDEA['UDEA_authors']=kk

In [67]:
UDEA.UDEA_authors.dropna().shape,UDEA.shape

((10901,), (15700, 181))

((10963,), (15704, 181))

In [68]:
aunly.shape

(1273, 2)

(1461, 2)

In [69]:
if not aunly.empty:
    print(aunly.drop_duplicates('tmp_author').shape)

(1273, 2)


In [70]:
if not aunly.empty:
    aunly.to_json('UDEA_authors_with_WOS_info.json')

In [71]:
RECOVER=False
if RECOVER:
    if os.path.exists('UDEA_authors_with_WOS_info.json' ):
        aunly=pd.read_json('UDEA_authors_with_WOS_info.json')
    else:
        aunly=drive_files.read_drive_json('UDEA_authors_with_WOS_info.json')

In [72]:
if UDEA['UDEA_authors'].dropna().shape[0]:
    UDEA.to_json('UDEAtmp.json')

In [73]:
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

In [74]:
UDEA.to_json('WOS_SCI_SCP_PTJ_CTR.json.gz',compression='gzip')

In [75]:
if 'UDEA_autores' in UDEA.columns:
    print(UDEA[UDEA['UDEA_autores']==''].shape[0],UDEA['UDEA_autores'].dropna().shape[0])

0 10311


In [83]:
if 'UDEA_authors' in UDEA.columns:
    print(UDEA[UDEA['UDEA_authors']==''].shape[0],UDEA['UDEA_authors'].dropna().shape[0])

0 10901


In [77]:
print 1

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-77-c94594b6b28f>, line 1)

In [ ]:
check_quality(UDEA)

## Exactly as before: Used only if 'UDEA_authors' is empty! 

In [ ]:
if RECOVER:
    UDEA=pd.read_json('WOS_SCI_SCP_PTJ_CTR.json.gz',compression='gzip').reset_index(drop=True)
    aunly=drive_files.read_drive_json('UDEA_authors_with_WOS_info.json').reset_index(drop=True)

Same function that `get_UDEA_authors??` but for apply instead of combine

In [ ]:
def build_institutional_authors(x,author_df,x_author_key='WOS_author',x_affiliation_key='affiliation',
                                        author_key='WOS_author',
                                        affiliation_key='WOS_affiliation'):
    '''
    Same function that 
    get_UDEA_authors?? 
    but for apply instead of combine
    '''
    if type(x)!=list:
        return None
    ll=[]
    for j in range(len(x)):
        
                                #author_WOS→affiliation always have single affiliation
        kk=find_author_affiliation(x[j].get(x_author_key),x[j].get(x_affiliation_key)[0],
                                        author_df=author_df,
                                        author_key=author_key,
                                        affiliation_key=affiliation_key,
                                        ratio=0.9 )
        if kk:
            ll.append(kk)
    if not ll:
        ll=None
    return ll

In [ ]:
if not UDEA['UDEA_authors'].dropna().shape[0]:
    UDEA['UDEA_authors']=UDEA.authors_WOS.apply(lambda l: build_institutional_authors(l,aunly) )

## Experimental: Try a similiraty merge with the remaining entries
### TODO: Change by new function
### TODO: Fix missing `authors_WOS`

In [85]:
UDEA_YES=UDEA[~UDEA['UDEA_authors'].isna()].reset_index(drop=True)

In [86]:
UDEA_YES.shape

(10901, 181)

In [87]:
import fuzzywuzzy.process as fwp
from fuzzywuzzy import fuzz
UDEA_NOT=UDEA[UDEA['UDEA_authors'].isna()].reset_index(drop=True)
df2=aunly.copy()
df2=pd.DataFrame( list( df2['UDEA_authors'].values ) )
df2['UDEA_authors']=aunly['UDEA_authors']
contents=df2[['WOS_author','WOS_affiliation','UDEA_authors']].reset_index(drop=True)
contents['WOS_author']=contents['WOS_author'].astype(str)
contents['WOS_affiliation']=contents['WOS_affiliation'].astype(str)

In [88]:
print( UDEA_NOT['authors_WOS'].loc[0][0].get('WOS_author'),
      fwp.extractOne(  UDEA_NOT['authors_WOS'].loc[0][0].get('WOS_author'),
                     contents['WOS_author'],scorer=fuzz.partial_ratio  ) )

Maria Carrillo-Bonilla, Lina ("['Carrillo-Bonilla, Lina M.', 'Carrillo Bonilla, Lina Maria', 'CARRILLO, Lina M.']", 86, 841)


In [89]:
dfnot=UDEA_NOT.copy()
dfnot=dfnot.reset_index(drop=True)

In [107]:
#for i in range(20):
#l=dfnot['authors_WOS'].loc[i]
def json_fuzzy_merge_full(l,contents,right_target='UDEA_authors',
                       left_on='WOS_author',extra_left_on='affiliation',
                       right_on='WOS_author',extra_right_on='WOS_affiliation',
                       cutoff=95,cutoff_extra=65,scorer=fuzz.partial_ratio):
    newl=[]
    for d in l:
        au=d.get(left_on)
        aff=d.get(extra_left_on)
        # Do not need to be string
        r=fwp.extractOne(au,contents[right_on],scorer=scorer)
        if r[1]>=cutoff:
            raf=scorer( aff, contents.loc[r[2],extra_right_on]  )
            if raf>=cutoff_extra:
                mthchedd=contents.loc[r[2],right_target]
                mthchedd['from_author_WOS_WOS_author']=au
                newl=newl+[ mthchedd   ]
    if newl:
        return newl
    else:
        return None

In [108]:
#for i in range(20):
#l=dfnot['authors_WOS'].loc[i]
#93,70
from IPython.display import clear_output
def json_fuzzy_merge(l,so,UDEA,contents,right_target='UDEA_authors',
                       left_on='WOS_author',extra_left_on='affiliation',
                       right_on='WOS_author',extra_right_on='WOS_affiliation',
                       extra_extra_right_on='full_name',
                       cutoff=93,cutoff_extra=70,scorer=fuzz.token_set_ratio,
                       DEBUG=False):
    newl=[]
    for d in l:
        clear_output(wait=True)
        AUTHOR=False
        AFFILIATION=False
        JOURNAL=True

        dfraf=pd.DataFrame()
        au=d.get(left_on)
        aff=d.get(extra_left_on)[0]
        Q=1
        # Try match author to a good degree
        rau=fwp.extractOne(au,contents[right_on].apply(pd.Series).stack().unique(),scorer=scorer)
        if DEBUG: print(1,rau)
        if rau[1]>=cutoff:
            AUTHOR=True
        #Try match author with less quality: Q
        else:
            rau=fwp.extractOne(au,contents[right_on].apply(pd.Series).stack().unique(),
                       scorer=fuzz.partial_token_sort_ratio)
            if DEBUG: print(1.1,rau)            
            if rau and rau[1]>=cutoff:
                Q=Q-0.1
                AUTHOR=True
        if DEBUG: print(1.2,'AUTHOR:',AUTHOR)                            
        if AUTHOR:
            dfraf=contents[contents[right_on].apply( lambda l: rau[0] in l )
                                ].reset_index(drop=True)
            raf=fwp.extractOne(aff,dfraf[extra_right_on].loc[0],scorer=fuzz.ratio)
            if DEBUG: print(2,raf)
            if raf and raf[1]>=cutoff_extra:
                AFFILIATION=True
            else:
                Q=Q-0.1
                raf=fwp.extractOne(aff,dfraf[extra_right_on].loc[0],
                                   scorer=fuzz.partial_token_set_ratio)
                if DEBUG: print(2.1,raf)
                if raf and raf[1]>=cutoff_extra:
                    AFFILIATION=True

        if DEBUG: print(2.2,'AFFILIATION:',AFFILIATION,'Q:',Q)                
        if Q<1:
            cutoff_so=50
            if Q<0.9:
                cutoff_so=60
            if not dfraf.empty:
                full_name=dfraf[right_target].loc[0].get(
                        extra_extra_right_on)
                if full_name:
                    kkk=UDEA[UDEA['UDEA_nombre'].str.contains(full_name)
                                ].reset_index(drop=True)
                    rso=fwp.extractOne( so,   kkk.SO, scorer=scorer)
                    if not rso:
                        JOURNAL=False
                    elif rso[1]<cutoff_so:
                        JOURNAL=False
            else:
                JOURNAL=False
        if DEBUG: print(3,'JOURNAL',JOURNAL)                
        if AUTHOR and AFFILIATION and JOURNAL:
            mthchedd=dfraf.loc[0,right_target]
            mthchedd['from_author_WOS_WOS_author']=au
            newl=newl+[  mthchedd  ]            
            print('{} → {}'.format(au,newl[0][extra_extra_right_on]) ) 
    if newl:
        return newl
    else:
        return None

In [109]:
import pandas as pd
import swifter

In [ ]:
%time kk=dfnot['authors_WOS'].swifter.apply(lambda l: json_fuzzy_merge_full(l,contents,cutoff_extra=65))

In [ ]:
#%time kk=dfnot['authors_WOS'].apply(lambda l: json_fuzzy_merge(l,contents,cutoff_extra=65))

In [ ]:
%time kkk=dfnot['authors_WOS'].combine(dfnot['SO'],func=lambda l,so: json_fuzzy_merge(l,so,UDEA.fillna(''),contents) if type(l)==list else None)

In [100]:
kk.dropna().shape

(909,)

In [101]:
764+883

1647

In [ ]:
dfnot['UDEA_authors']=kk

In [ ]:
#dfnot[['UDEA_authors','authors_WOS']].dropna(subset=['UDEA_authors'])

In [ ]:
UDEA_NOT=dfnot.reset_index(drop=True)

In [ ]:
UDEA=UDEA_YES.append(UDEA_NOT).reset_index(drop=True)

In [ ]:
UDEA['UDEA_authors'].dropna().shape

Quality checks

UDEA_YES=UDEA[UDEA.UDEA_nombre!=''].reset_index(drop=True)
UDEA_NOT=UDEA[UDEA.UDEA_nombre==''].reset_index(drop=True)

UDEA_YES['Tipo']=UDEA_YES.Tipo.str.replace('([SW][CO][SIP])$',r'\1_UDEA')

UDEA=UDEA_YES.append(UDEA_NOT)
UDEA=UDEA.reset_index(drop=True)

UDEA[UDEA.Tipo.str.contains('UDEA')].shape

In [ ]:
UDEA.to_json('WOS_SCI_SCP_PTJ_CTR.json.gz',compression='gzip')

In [ ]:
check_quality(UDEA)

In [ ]:
print 1

## Try other approachs

In [ ]:
wp.merge_with_close_matches??

In [ ]:
%%writefile test.cfg
[FILES]
Sample_WOS.xlsx = 1--LJZ4mYyQcaJ93xBdbnYj-ZzdjO2Wq2
Sample_SCI.xlsx = 1-3a-hguQTk5ko8JRLCx--EKaslxGVscf
Sample_SCP.xlsx = 1-IAWlMdp2U-9L2jvZUio04ub1Ym3PX-H

In [ ]:
cib=wp.wosplus('test.cfg')
#cib.Debug=True
cib.load_biblio('Sample_WOS.xlsx')
cib.load_biblio('Sample_SCI.xlsx',prefix='SCI')
cib.load_biblio('Sample_SCP.xlsx',prefix='SCP')

In [ ]:
def get_close_matches_Levenshtein(
        word,
        possibilities,
        n=3,
        cutoff=0.6,
        full=False):
    '''Replaces difflib.get_close_matches with faster algortihm based on
       Levenshtein.ratio.
       HINT: Similarity increase significatively after lower() and unidecode()

       Refs: https://en.wikipedia.org/wiki/Levenshtein_distance
    '''
    import pandas as pd
    import Levenshtein
    if isinstance(possibilities, str):
        possibilities = [possibilities]
    rs = pd.DataFrame()
    MATCH = False
    for p in possibilities:
        similarity = Levenshtein.ratio(word, p)
        # print(word,'::',p,similarity)
        # sys.exit()
        if similarity >= cutoff:
            MATCH = True
            rs = rs.append({'similarity': similarity,
                            'match': p}, ignore_index=True)

    if MATCH:
        rs = rs.sort_values(
            'similarity', ascending=False).reset_index(drop=True)
        if full:
            return list(rs['match'][:n].values), list(
                rs['similarity'][:n].values)
        else:
            return list(rs['match'][:n].values)
    else:
        if full:
            return ([], 0)
        else:
            return []

In [ ]:
scorer=fuzz.ratio

In [ ]:
def get_close_matches_Levenshtein_new(
        word,
        possibilities,
        n=1,
        cutoff=0.6,
        full=False,
        scorer=fuzz.ratio): #
    r=fwp.extract(word,possibilities,scorer=scorer,limit=n)
    
    if r[0][1]/100.>cutoff:
        if full:
            return [t[0] for t in r],[t[1]/100. for t in r]
        else:
            return [t[0] for t in r]
    else:
        if full:
            return ([], 0)
        else:
            return []        

In [ ]:
def merge_with_close_matches_old(
        left,
        right,
        left_on='ST',
        right_on='UDEA_simple_título',
        left_extra_on='SO',
        right_extra_on='UDEA_nombre revista o premio',
        how='inner',
        n=1,
        cutoff=0.6,
        full=True,
        cutoff_extra=0.6):
    '''For each entry of the column: left_on of DataFrame left (cannot have empty fields),
       try to find the close match inside each row of right DataFrame, by comparing with
       the right_on entry of the row. When a row match is found, the full right row is appended
       to the matched row in the left DataFrame.
       If the similarity between the entries at left_on and right_on is less than 0.8,
       an additional check is performed between the entries left_extra_on and right_extra_on
       of the matched row.

       how implemented: inner and left (Default: inner)
    '''
    import numpy as np
    from unidecode import unidecode
    import pandas as pd
    # import sys #import globally
    # print(left[left_on][0])
    # sys.exit()
    words = left[left_on].str.lower().map(unidecode)
    possibilities = right[right_on].str.lower().map(unidecode)

    joined = pd.DataFrame()
    mi = np.array([])
    for i in left.index:
        if i % 100 == 0:
            print('.', end="")
        joined_series = left.loc[i]
        #joined_series=joined_series.append(pd.Series( {similarity_column:0} ))
        title, similarity = get_close_matches_Levenshtein(
            words[i], possibilities, n=n, cutoff=cutoff, full=full)
        # print(i,words[i],title,similarity) #cutuff 0.6 0.7 0.8 0.85 0.91 0.95
        # sys.exit()
        if title:
            mtch = right[possibilities == title[0]]
            # >=cutoff, e.g 0.65 0.95 0.81 0.86 0.9 0.96
            chk_cutoff = similarity[0]
            crosscheck = cutoff + 0.2  # 0.8 # e.g. 0.8 0.9 0.9 0.9 0.9 0.9
            if crosscheck >= 1:
                # force check if match worst than this (by experience)
                crosscheck = 0.95
            if chk_cutoff < crosscheck:  # e.g 0.65<0.8 0.95~<0.9 0.81~<0.0 0.86<0.9 0.91<~0.9 0.96~<0.9
                if get_close_matches_Levenshtein(unidecode(left[left_extra_on][i].lower()), [unidecode(
                        mtch[right_extra_on][mtch.index[0]].lower())], cutoff=cutoff_extra):  # cutoff=0.6
                    chk_cutoff = crosscheck + 0.1

            if chk_cutoff >= crosscheck:
                joined_series = joined_series.append(mtch.loc[mtch.index[0]])
                if how == 'outer':
                    mi = np.concatenate((mi, mtch.index.values))
                # joined_series[similarity_column]=similarity[0]

            #return joined_series
            if how == 'inner':
                joined = joined.append(joined_series, ignore_index=True)

        if (how == 'left' or 'outer'):
            joined = joined.append(joined_series, ignore_index=True)
    if how == 'outer':
        joined = joined.append(right.drop(
            right.index[list(mi.astype(int))]).reset_index(drop=True))
    return joined

def merge_with_close_matches_new(
        left,
        right,
        left_on='ST',
        right_on='UDEA_simple_título',
        left_extra_on='SO',
        right_extra_on='UDEA_nombre revista o premio',
        how='inner',
        n=1,
        cutoff=0.6,
        full=True,
        cutoff_extra=0.7):
    '''For each entry of the column: left_on of DataFrame left (cannot have empty fields),
       try to find the close match inside each row of right DataFrame, by comparing with
       the right_on entry of the row. When a row match is found, the full right row is appended
       to the matched row in the left DataFrame.
       If the similarity between the entries at left_on and right_on is less than 0.8,
       an additional check is performed between the entries left_extra_on and right_extra_on
       of the matched row.

       how implemented: inner and left (Default: inner)
    '''
    import numpy as np
    from unidecode import unidecode
    import pandas as pd
    # import sys #import globally
    # print(left[left_on][0])
    # sys.exit()
    words = left[left_on].str.lower().map(unidecode)
    possibilities = right[right_on].str.lower().map(unidecode)

    joined = pd.DataFrame()
    mi = np.array([])
    for i in left.index:
        if i % 100 == 0:
            print('.', end="")
        joined_series = left.loc[i]
        #joined_series=joined_series.append(pd.Series( {similarity_column:0} ))
        title, similarity = get_close_matches_Levenshtein_new(
            words[i], possibilities, n=n, cutoff=cutoff, full=full)
        # print(i,words[i],title,similarity) #cutuff 0.6 0.7 0.8 0.85 0.91 0.95
        # sys.exit()
        if title:
            mtch = right[possibilities == title[0]]
            # >=cutoff, e.g 0.65 0.95 0.81 0.86 0.9 0.96
            chk_cutoff = similarity[0]
            crosscheck = cutoff + 0.2  # 0.8 # e.g. 0.8 0.9 0.9 0.9 0.9 0.9
            if crosscheck >= 1:
                # force check if match worst than this (by experience)
                crosscheck = 0.95
            if chk_cutoff < crosscheck:  # e.g 0.65<0.8 0.95~<0.9 0.81~<0.0 0.86<0.9 0.91<~0.9 0.96~<0.9
                if get_close_matches_Levenshtein_new(unidecode(left[left_extra_on][i].lower()), [unidecode(
                        mtch[right_extra_on][mtch.index[0]].lower())], cutoff=cutoff_extra):  # cutoff=0.6
                    chk_cutoff = crosscheck + 0.1

            if chk_cutoff >= crosscheck:
                joined_series = joined_series.append(mtch.loc[mtch.index[0]])
                if how == 'outer':
                    mi = np.concatenate((mi, mtch.index.values))
                # joined_series[similarity_column]=similarity[0]

            #return joined_series
            if how == 'inner':
                joined = joined.append(joined_series, ignore_index=True)

        if (how == 'left' or 'outer'):
            joined = joined.append(joined_series, ignore_index=True)
    if how == 'outer':
        joined = joined.append(right.drop(
            right.index[list(mi.astype(int))]).reset_index(drop=True))
    return joined

In [ ]:
cib.biblio['WOS']=UDEA.sample(500).reset_index(drop=True).copy().fillna('')
cib.biblio['SCI']=SIU[0:100].copy().fillna('')

In [ ]:
tmp=drive_files.load_biblio('produccion_fecha_vig_2003_2018.xlsx',prefix='UDEA')

In [ ]:
drive_files.biblio['UDEA']

In [ ]:
%time kkold=merge_with_close_matches_old(cib.biblio['WOS'],cib.biblio['SCI'].drop('Tipo',axis='columns'),left_on='TI',right_on='SCI_TI',right_extra_on='SCI_SO',how='left')


In [ ]:
%time kknew=merge_with_close_matches_new(cib.biblio['WOS'],cib.biblio['SCI'].drop('Tipo',axis='columns'),left_on='TI',right_on='SCI_TI',right_extra_on='SCI_SO',how='left')


In [ ]:
kkold.shape,kknew.shape

In [ ]:
(kkold['SCI_TI'].apply(lambda s: s if s else pd.np.nan).dropna().shape,
 kknew['SCI_TI'].apply(lambda s: s if s else pd.np.nan).dropna().shape)

In [ ]:
kknew[['TI','SCI_TI']].dropna()

In [ ]:
kkold[['TI','SCI_TI']].dropna()

In [ ]:
import time 

In [ ]:
s=time.time()
kkold=merge_with_close_matches_old(cib.biblio['WOS'][['TI','SO']],drive_files.biblio['UDEA'][['UDEA_título',
                                                            'UDEA_nombre revista o premio']],
                            left_on='TI',left_extra_on='SO',right_on='UDEA_título',
                            right_extra_on='UDEA_nombre revista o premio',how='left')
print(time.time()-s)

In [ ]:
s=time.time()
kknew=merge_with_close_matches_new(cib.biblio['WOS'][['TI','SO']],drive_files.biblio['UDEA'][['UDEA_título',
                                                            'UDEA_nombre revista o premio']],
                            left_on='TI',left_extra_on='SO',right_on='UDEA_título',
                            right_extra_on='UDEA_nombre revista o premio',how='left')
print(time.time()-s)

In [ ]:
..................

In [ ]:
(kkold['UDEA_título'].apply(lambda s: s if s else pd.np.nan).dropna().shape,
 kknew['UDEA_título'].apply(lambda s: s if s else pd.np.nan).dropna().shape)